In [1]:
import numpy as np

from lcc.dataset import create_lcc_dataset_k1_t1_scalar

from probabilistic_classifier.dataset import create_knn_sampling_joint_cond_marginal_dataset
from probabilistic_classifier.estimate import estimate_mi_for_binary_classification
from probabilistic_classifier.train import train_binary_classifier_v2

In [2]:
# create the basis dataset
prime = 5
data_range = 2
num_of_samples = 800000
weight = 1
dataset = create_lcc_dataset_k1_t1_scalar(prime, data_range, num_of_samples, weight)

KeyboardInterrupt: 

In [ ]:
dataset = np.concatenate([dataset[:, :3], dataset[:, 4:]], axis=1)

In [ ]:
# create the joint and marginal datasets
num_of_neighbors = 4
x_idx, y_idx, z_idx = [0, 1], [3, 4, 5], [2]
joint_data, joint_label, marginal_data, marginal_label = create_knn_sampling_joint_cond_marginal_dataset(dataset, num_of_neighbors, x_idx, y_idx, z_idx)
data, label = np.concatenate([joint_data, marginal_data]), np.concatenate([joint_label, marginal_label])
randomize_idx = np.random.permutation(np.arange(2 * num_of_samples))
data, label = data[randomize_idx], label[randomize_idx]

In [ ]:
# define model parameters
num_input_features = len(x_idx) + len(y_idx) + len(z_idx)
hidden_size_arr = [256, 256, 256]
lr = 0.001

In [ ]:
num_of_outer_iteration = 20
num_of_inner_iteration = 50
batch_size = 4096

# iterate over many times
outer_running_loss = []
outer_running_loss_avg = []
ldr_estimations = []
dv_estimations = []
nwj_estimations = []

for outer_iter in range(num_of_outer_iteration):
    print('################################################################')
    model, inner_running_loss, inner_running_loss_avg, num_of_joint, num_of_marginal = train_binary_classifier_v2(data, label, num_input_features, hidden_size_arr, lr, num_of_inner_iteration, batch_size, outer_iter, save_avg=200, print_progress=True)
    outer_running_loss.append(inner_running_loss)
    outer_running_loss_avg.append(inner_running_loss_avg)
    
    ## estimate cmi
    curr_ldr, curr_dv, curr_nwj = estimate_mi_for_binary_classification(model, joint_data, num_of_joint, marginal_data, num_of_marginal)
    print('trial: {}, ldr: {}, dv: {}, nwj: {}'.format(outer_iter + 1, curr_ldr.item(), curr_dv.item(), curr_nwj.item()))
    print('################################################################\n')
    ldr_estimations.append(curr_ldr.item())
    dv_estimations.append(curr_dv.item())
    nwj_estimations.append(curr_nwj.item())
    
print('final estimations:\n\tldr: {}\n\tdv: {}\n\tnwj: {}'.format(np.mean(ldr_estimations), np.mean(dv_estimations), np.mean(nwj_estimations)))